## 1. 웹스크래핑 연습문제

* user-agent 요청헤더를 반드시 설정해야 한다.

In [ ]:
# requests 라이브러리 설치여부 확인
!pip show requests
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

In [2]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

### 질문 1: Daum 뉴스기사 제목 스크래핑하기

In [ ]:
# 다음 경제 뉴스 URL
url = 'https://news.daum.net/economy'
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}

# requests 의 get() 함수 호출하기 
res = requests.get(url, headers=req_header)
print(type(res))
print(res.status_code)
# 응답(response)이 OK 이면 응답 (response)에서 text 추출
# BeautifulSoup 객체 생성
if res.ok :
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    # CSS 선택자를 사용해서 a tag 목록 가져오기
    a_tags = soup.select("ul.list_newsheadline2 a[href*='https://v.daum.net/v/']")
    print(type(a_tags), len(a_tags))
    # <a> 태그 리스트 순회하기    
    for a_tag in a_tags :
        link = a_tag['href']
        strong_tag = a_tag.select_one('div.cont_thumb strong.tit_txt')
        title = strong_tag.text
        
        print(link, title)  
else :
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')

### 질문2:  여러개의 section 중 하나를 선택해서 url에서 뉴스기사의 링크와 제목을 출력하는 코드를 함수로 작성하기

In [59]:
import requests
from bs4 import BeautifulSoup

section_dict = {'기후/환경':'climate','사회':'society','경제':'economy','정치':'politics',\
             '국제':'world','문화':'culture','생활':'life','IT/과학':'tech','인물':'people'}

def print_news(section_name):  #print_new(life,'생활') 
    sid = section_dict.get(section_name)
    url = f'https://news.daum.net/{sid}'
    print(f'======> {url} {section_name} 뉴스 <======')
    req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}
    res = requests.get(url, headers=req_header)
    if res.ok :
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        # CSS 선택자를 사용해서 a tag 목록 가져오기
        a_tags = soup.select("ul.list_newsheadline2 a[href*='https://v.daum.net/v/']")
        # <a> 태그 리스트 순회하기    
        for a_tag in a_tags :
            link = a_tag['href']
            strong_tag = a_tag.select_one('div.cont_thumb strong.tit_txt')
            title = strong_tag.text
            
            print(link)
            print(title)
            # 하나의 기사만 보고싶히면 break 사용
            break
    else :
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')

In [ ]:
print_news('경제')
print_news('국제')

### 2-1. Nate 뉴스기사 제목 스크래핑하기

In [ ]:
import requests
import os
# 육아일기 73회차
req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=812354&no=208&week=sun'
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_3.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기 
    res = requests.get(img_url, headers=req_header)
    print(res.status_code)        
    # binary 응답 데이터 가져오기
    img_data = res.content    
    # url에서 파일명만 추출하기
    file_name = os.path.basename(img_url)
    print(file_name)        
    # binday data를 file에 write하기
    with open(file_name,'wb') as file:
        print(f'Writing to {file_name}({len(img_data):,} bytes)')
        file.write(img_data)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.



200


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청


In [ ]:
# 캡차 이미지 요청


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



Error Code: 403


In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': '',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'


# requests get(url, params, headers) 요청 

# json() 함수로 응답 결과 가져오오기
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

# 'data/nhnblog.txt' 파일 생성하기
